# Optimization with Linear Regression, SVM, and COBYLA

This notebook demonstrates an optimization workflow involving multiple linear regression, polynomial regression using Support Vector Machines (SVM), and optimization techniques including Simplex and COBYLA.

```python


In [14]:
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import linprog, minimize
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

# Step 1: Multiple Linear Regression
In this step, we perform multiple linear regression on the provided data to obtain coefficients and construct the regression equation.

In [15]:
def linear_regression_summary(csv_file_path):
    data = pd.read_csv(csv_file_path)
    y = data['z']
    X = data[['x1', 'x2']]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    coefficients = model.params
    regression_equation = f"z = {coefficients[0]:.4f} + {coefficients[1]:.4f} * x1 + {coefficients[2]:.4f} * x2"
    return coefficients, regression_equation

# Step 2: Simplex Method for Linear Optimization
We use the Simplex method to maximize the linear objective function derived from the coefficients obtained in Step 1. The constraints are included in the optimization process.

In [16]:
def simplex_optimization(coefficients):
    c = [-coefficients[1], -coefficients[2]]  # Minimizing -z is equivalent to maximizing z
    A = [[2, 5]]
    b = [98]
    bounds = [(0, None), (0, None)]
    
    result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='simplex')
    
    if result.success:
        x1, x2 = result.x
        z = -result.fun  # Revert the sign to get the original maximized value
        return x1, x2, z
    else:
        raise ValueError("Simplex method failed to find a solution")

# Step 3: SVM for Polynomial Regression
Here, we fit an SVM model with polynomial features to the data. This step also extracts the polynomial coefficients and intercept needed for further optimization.

In [17]:
def fit_svm_polynomial(csv_file_path):
    data = pd.read_csv(csv_file_path)
    y = data['z']
    X = data[['x1', 'x2']]

    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)

    model = make_pipeline(StandardScaler(), SVR(kernel='linear', C=1, epsilon=0.1))
    model.fit(X_poly, y)

    lin_reg = make_pipeline(PolynomialFeatures(degree=2, include_bias=True), StandardScaler(), SVR(kernel='linear', C=1, epsilon=0.1))
    lin_reg.fit(X, y)
    intercept = lin_reg.named_steps['svr'].intercept_
    coefficients = lin_reg.named_steps['svr'].coef_
    return coefficients, intercept, poly


# Step 4: COBYLA Optimization
We use the COBYLA method to maximize the polynomial function derived from the SVM model. The bounds and constraints are included in this optimization step.

In [18]:
def cobyla_maximization(coefficients, intercept, poly, initial_guess):
    def objective_function(x):
        x_poly = poly.transform([x])
        return -(intercept + sum(c * x_poly[0, i] for i, c in enumerate(coefficients)))

    bounds = [(0, None), (0, None)]
    constraint_function = lambda x: 98 - 2*x[0] - 5*x[1]
    constraints = [{'type': 'ineq', 'fun': constraint_function}]

    result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints, method='COBYLA')
    
    if result.success:
        maximized_x1, maximized_x2 = result.x
        maximized_z = -result.fun
        return maximized_x1, maximized_x2, maximized_z
    else:
        print("Optimization failed. Here is the result:")
        print(result)  # Print the full result to understand the failure reason
        raise ValueError("COBYLA method failed to find a solution")


# Example Usage
Finally, we demonstrate how to use these functions with a sample CSV file to perform linear regression, optimization with Simplex and COBYLA, and print the polynomial equation.

In [19]:
csv_file_path = "equation_data_with_bias.csv"  # Provide the correct path to your CSV file

# Step 1: Linear regression to get coefficients
coefficients, equation_string = linear_regression_summary(csv_file_path)
print("Linear Regression Equation:")
print(equation_string)

# Step 2: Simplex optimization
x1_simplex, x2_simplex, z_simplex = simplex_optimization(coefficients)
print("Simplex Optimization Result:")
print(f"x1 = {x1_simplex:.4f}, x2 = {x2_simplex:.4f}, z = {z_simplex:.4f}")

# Step 3: Fit SVM with polynomial features
svm_coefficients, intercept, poly = fit_svm_polynomial(csv_file_path)

# Step 4: Maximize the polynomial function using COBYLA
try:
    maximized_x1_cobyla, maximized_x2_cobyla, maximized_z_cobyla = cobyla_maximization(svm_coefficients, intercept, poly, initial_guess=[x1_simplex, x2_simplex])
    print("Maximized values using COBYLA:")
    print(f"x1 = {maximized_x1_cobyla:.4f}")
    print(f"x2 = {maximized_x2_cobyla:.4f}")
    print(f"Maximized z = {maximized_z_cobyla:.4f}")
except ValueError as e:
    print(f"Error: {e}")

Linear Regression Equation:
z = -1621.0695 + 963.4735 * x1 + -657.5960 * x2
Simplex Optimization Result:
x1 = 49.0000, x2 = 0.0000, z = 47210.2017
Maximized values using COBYLA:
x1 = 49.0000
x2 = 0.0000
Maximized z = 2925.6275


/var/folders/ww/qjh9zpks52d4vsjxc4bg379h0000gn/T/ipykernel_3212/270654717.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  regression_equation = f"z = {coefficients[0]:.4f} + {coefficients[1]:.4f} * x1 + {coefficients[2]:.4f} * x2"
/var/folders/ww/qjh9zpks52d4vsjxc4bg379h0000gn/T/ipykernel_3212/3928591052.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c = [-coefficients[1], -coefficients[2]]  # Minimizing -z is equivalent to maximizing z
/var/folders/ww/qjh9zpks52d4vsjxc4bg379h0000gn/T/ipykernel_3212/3928591052.py:7: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Ple

# Appendix 6: Counterexample Demonstrating COBYLA Failure with a Random Initial Guess
This section demonstrates a scenario where the COBYLA optimization method fails to converge when starting from an arbitrary initial guess. However, COBYLA successfully converges when using the initial guess derived from the Simplex method solution. You can change the initial guesses which is out of bound and most of them will fail to converge making them more than 20 examples.

In [20]:
objective_function = lambda x: -(coefficients[0] * x[0]**2 + coefficients[1] * x[1]**2 + coefficients[2] * x[0] * x[1])
bounds = [(0, None), (0, None)]
constraint_function = lambda x: 98 - 2*x[0] - 5*x[1]
initial_guess_fail = [-100, -100]
result_fail = minimize(objective_function, initial_guess_fail, method='COBYLA', bounds=bounds, constraints=constraints)
print("Optimization method used:", result_fail.message)
if result_fail.success:
    maximized_x1_fail, maximized_x2_fail = result_fail.x
    maximized_z_fail = -result_fail.fun  
    steps_fail = result_fail.nfev

    print("Maximized values with an initial guess that may fail:")
    print(f"x1 = {maximized_x1_fail:.4f}")
    print(f"x2 = {maximized_x2_fail:.4f}")
    print(f"Maximized z = {maximized_z_fail:.4f}")
    print(f"Number of steps taken: {steps_fail}")
else:
    print("COBYLA method failed to converge with the initial guess.")

Optimization method used: Maximum number of function evaluations has been exceeded.
COBYLA method failed to converge with the initial guess.


/Users/rishav/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minimize.py:569: RuntimeWarning: Method COBYLA cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
/var/folders/ww/qjh9zpks52d4vsjxc4bg379h0000gn/T/ipykernel_3212/3910368584.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  objective_function = lambda x: -(coefficients[0] * x[0]**2 + coefficients[1] * x[1]**2 + coefficients[2] * x[0] * x[1])
